In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)

output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value



In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("omni-research/DREAM-1K",cache_dir = './video')

/home/swq/miniconda3/envs/Qwen25vl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 1000 examples [00:00, 43215.74 examples/s]


In [7]:
ds

DatasetDict({
    test: Dataset({
        features: ['description', 'source', 'n_events', 'n_shots', 'events', 'video_file', 'n_subjects', 'idx', 'duration'],
        num_rows: 1000
    })
})

In [8]:
ds['test'][-10:]

{'description': ['The video shows astronauts in the cockpit of a spacecraft, manipulating several buttons on the control panel which results in the spacecraft shaking violently. An astronaut tries to stabilize the craft by adjusting a control lever, but their terrified and scared expressions indicate the dire state of the situation.',
  "In this video, an individual donned in a brown suit jacket is seen obstinately attempting to operate a television using a remote control. Another individual, clad in a black suit jacket, repeatedly advises him that the remote does not control the TV. Despite this, the person continues to depress the remote's buttons. Eventually, a transparent swan ornament, perched atop the TV, is ejected by a suddenly-opening cover. The swan ornament collides with a hanging glass photo frame, containing a picture of a family – two people and a white dog – causing the glass to shatter.",
  "In this video, a person wearing a brown suit is observing an oil painting hung 

In [9]:
import json
import random
from tqdm import tqdm

# 假设 ds 是你的 Hugging Face DatasetDict 对象
# 例如: ds = load_dataset('your_dataset_name')

# 首先选择 'test' 拆分
ds_test = ds['test']

# 打乱数据以确保随机性，并设置种子以便结果可复现
ds_test = ds_test.shuffle(seed=42)

# 只选择前400条数据
ds_test = ds_test.select(range(400))

# 计算分割点
split_point = int(len(ds_test) * 0.8)

# 分割数据集
train_data = ds_test.select(range(split_point))
val_data = ds_test.select(range(split_point, len(ds_test)))

def format_entries(dataset):
    formatted_entries = []
    for entry in tqdm(dataset, desc="Processing entries", unit="entry"):
        formatted_entry = {
            "messages": [
                {
                    "content": "<video>Please describe this video, mention who are the characters in it, what they did, and what is the background?",
                    "role": "user"
                },
                {
                    "content": f"Description: {entry['description']}\nEvents:\n" + "\n".join(entry['events']),
                    "role": "assistant"
                }
            ],
            "videos": [
                f"video/DREAM-1K_videos/{entry['video_file'].split('/')[-1]}"
            ]
        }
        formatted_entries.append(formatted_entry)
    return formatted_entries

# 格式化并保存训练集
formatted_train_data = format_entries(train_data)
with open('train_output.json', 'w', encoding='utf-8') as f:
    json.dump(formatted_train_data, f, ensure_ascii=False, indent=4)

# 格式化并保存验证集
formatted_val_data = format_entries(val_data)
with open('test_output.json', 'w', encoding='utf-8') as f:
    json.dump(formatted_val_data, f, ensure_ascii=False, indent=4)

print("Data has been successfully written to train_output.json and val_output.json")

Processing entries: 100%|██████████| 80/80 [00:00<00:00, 8327.19entry/s]

Data has been successfully written to train_output.json and val_output.json
